<center>
<img src='./img/nsidc_logo.png'/>
    
# **Accessing and working with ICESat-2 data in the cloud**
    
</center>
    

## **1. Tutorial Overview**

**Note: This is an updated version of the notebook that was presented to the NSIDC DAAC User Working Group in May 2022**

This notebook demonstrates searching for cloud-hosted ICESat-2 data and directly accessing Land Ice Height (ATL06) granules from an Amazon Compute Cloud (EC2) instance using the `earthaccess` package. NASA data "in the cloud" are stored in Amazon Web Services (AWS) Simple Storage Service (S3) Buckets. **Direct Access** is an efficient way to work with data stored in an S3 Bucket when you are working in the cloud. Cloud-hosted granules can be opened and loaded into memory without the need to download them first. This allows you take advantage of the scalability and power of cloud computing. 

The Amazon Global cloud is divided into geographical regions.  To have direct access to data stored in a region, our compute instance - the cloud server we are running our code on - must be in the same region as the data.  This is a fundamental concept of _analysis in place_. **NASA cloud-hosted data is in Amazon Region us-west2.  So your compute instance must also be in us-west2.** If we wanted to use data stored in another region, to use direct access for that data, we would start a compute instance in that region.

As an example data  collection, we use ICESat-2 Land Ice Height (ATL06) over the Juneau Icefield, AK, for March 2003. ICESat-2 data granules, including ATL06, are stored in HDF5 format. We demonstrate how to open an HDF5 granule and access data variables using `xarray`. Land Ice Heights are then plotted using `hvplot`. 

`earthaccess` is a package developed by Luis Lopez (NSIDC developer) to allow easy search of the NASA Common Metadata Repository (CMR) and download of NASA data collections.  It can be used for programmatic search and access for both _DAAC-hosted_ and _cloud-hosted_ data. It manages authenticating using Earthdata Login credentials which are then used to obtain the S3 tokens that are needed for S3 direct access. https://github.com/nsidc/earthaccess


### **Credits**

The notebook was created by Andy Barrett, NSIDC, updated by Jennifer Roebuck, NSIDC, and is based on notebooks developed by Luis Lopez and Mikala Beig, NSIDC.

For questions regarding the notebook, or to report problems, please create a new issue in the [NSIDC-Data-Tutorials repo](https://github.com/nsidc/NSIDC-Data-Tutorials/issues).

### **Learning Objectives**

By the end of this demonstration we will be able to:  
1. use `earthaccess` to search for ICESat-2 data using spatial and temporal filters and explore search results;  
2. open data granules using direct access to the ICESat-2 S3 bucket;  
3. load a HDF5 group into an `xarray.Dataset`;  
4. visualize the land ice heights using `hvplot`.  

### **Prerequisites**

1. We will be accessing data in Amazon Web Services (AWS) Simple Storage Service (S3) buckets. This requires an AWS account, for details on how to do set one up see [here](https://nsidc.org/data/user-resources/help-center/nasa-earthdata-cloud-data-access-guide#anchor-1).
2. **NASA cloud-hosted data is in Amazon Region us-west2. So we also need an EC2 instance in the us-west-2 region.** Details on how to set an instance up can be found [here](https://nsidc.org/data/user-resources/help-center/nasa-earthdata-cloud-data-access-guide#anchor-1).
3. An Earthdata Login is required for data access. If we don't have one, we can register for one [here](https://urs.earthdata.nasa.gov/).
4. Current recommended practice for authentication is to create a .netrc file in our home directory following [these instructions](https://nsidc.org/support/how/how-do-i-programmatically-request-data-services) (Step 1) and to use the .netrc file for authentication when required for data access during the tutorial.
5. The *nsidc-tutorials* environment is setup and activated. This [README](https://github.com/nsidc/NSIDC-Data-Tutorials/blob/main/README.md) has setup instructions.

### **Example of end product** 
At the end of this tutorial, the following figure will be generated:
<center>
<img src='./img/atl06_land_ice_heights_plot.png'/>
</center>

### **Time requirement**

Allow approximately 20 minutes to complete this tutorial.

## **2. Tutorial steps**

## Import Packages

The first step in any `python` script or notebook is to import packages.  In this tutorial requires the following pacakges:
- `earthaccess`, which enables Earthdata Login authentication and retrieves AWS credentials; enables collection and granule searches; and S3 access;
- `xarray`, used to load data;
- `hvplot`, used to visualize land ice height data.

We are going to import the whole `earthaccess` package.

We will also import the whole `xarray` package but use a standard short name `xr`, using the `import <package> as <short_name>` syntax.  We could use anything for a short name but `xr` is an accepted standard that most `xarray` users are familiar with.

We only need the `xarray` module from `hvplot` so we import that using the `import <package>.<module>` syntax.

In [ ]:
# For nice printing
from pprint import pprint

# For searching NASA data
import earthaccess

# For reading data, analysis and plotting
import xarray as xr
import hvplot.xarray

## Authenticate

The first step is to get the correct authentication that will allow us to get _cloud-hosted_ ICESat-2 data.  This is all done through Earthdata Login.  The `login` method also gets the correct AWS credentials.

Login requires your Earthdata Login username and password.  This can be entered using an Environment Variable - the default strategy - or using either a`.netrc` file or interactively.  A `.netrc` file is a text file located in our home directory that contains login information for remote machines.  If we don't have a `.netrc` file `login` can create one for us.

```
earthaccess.login(strategy='interactive', persist=True)
```

In [ ]:
auth = earthaccess.login(strategy='interactive')

## Search for ICESat-2 Collections

`earthaccess` leverages the Common Metadata Repository (CMR) API to search for collections and granules.  [Earthdata Search](https://search.earthdata.nasa.gov/search) also uses the CMR API.

We can use the `keyword` method for `collection_query` to search for ICESat-2 collections. 

In [ ]:
Query = earthaccess.collection_query().keyword('ICESat-2')

The `hits()` method can be used to find out how many collections (both _DAAC-hosted_ and _cloud-hosted_) we found.

In [ ]:
Query.hits()

We can see what these data collections are by _getting_ `ShortName` and `Versions`.  In this case, we'll just grab the first 10 results and print them out.

In [ ]:
collections = Query.fields(['ShortName', 'Version']).get(10)
print(collections)

The `get`, used above, returns a list of python dictionaries containing metadata `meta` and the two requested attribute fields from the Unified Metadata Model (UMM) entry for the collection `umm`.  The `meta` entry for each collection contains the following information.

- `concept-id`, which is a unique id for the collection.  We'll use the `concept_id` to search for data granules.
- `granule-count`, the number of data granules in the collection. 
- `provider-id`, the id for DAAC responsible for the collection.  This information is also part of the `concept-id`.

For the ICESat-2 search results there is a provider-id; `NSIDC_ECS` and `NSIDC_CPRD`. `NSIDC_ECS` which is for the _on-prem_ collections and `NSIDC_CPRD` is for the _cloud-hosted_ collections. 

The `umm` fields are `ShortName` and `Version`.  For ICESat-2, `ShortNames` are generally how different products are referred to.

| ShortName | Product Description |
|:-----------:|:---------------------|
| ATL03 | ATLAS/ICESat-2 L2A Global Geolocated Photon Data |
| ATL06 | ATLAS/ICESat-2 L3A Land Ice Height |
| ATL07 | ATLAS/ICESat-2 L3A Sea Ice Height |
| ATL08 | ATLAS/ICESat-2 L3A Land and Vegetation Height |
| ATL09 | ATLAS/ICESat-2 L3A Calibrated Backscatter Profiles and Atmospheric Layer Characteristics |
| ATL10 | ATLAS/ICESat-2 L3A Sea Ice Freeboard |
| ATL11 | ATLAS/ICESat-2 L3B Slope-Corrected Land Ice Height Time Series |
| ATL12 | ATLAS/ICESat-2 L3A Ocean Surface Height |
| ATL13 | ATLAS/ICESat-2 L3A Along Track Inland Surface Water Data |

### Search for cloud-hosted data
For most collections, to search for only data in the cloud, the `cloud_hosted` method can be used. 

In [ ]:
Query = earthaccess.collection_query().keyword('ICESat-2').cloud_hosted(True)
print(Query.hits())
collections = Query.fields(['ShortName', 'Version']).get(10)
print(collections)

## Search a data set using spatial and temporal filters 

As an example of a search using spatial and temporal filters, we'll search for ATL06 granules over the Juneau Icefield, AK, for March 2020.

The ATL06 version 005 collection is identified by the `concept_id` C2153572614-NSIDC_CPRD.  The temporal range is identified with standard date strings, and latitude-longitude corners of a bounding box is specified.  Polygons and points, as well as shapefiles can also be specified.

In [ ]:
Query = earthaccess.granule_query().concept_id(
    'C2153572614-NSIDC_CPRD'
).temporal(
    "2020-03-01", "2020-03-30"
).bounding_box(
    -134.7,58.9,-133.9,59.2)

As before, we can use `Query.hits()` to find the number of granules that match our search. 

In [ ]:
Query.hits()

We'll get metadata for these 4 granules and display it.  The rendered metadata shows a download link, granule size and two images of the data.  _This doesn't render so well on this Jupyter Hub._

The download link is `https` and can be used download the granule to your local machine.  This is similar to downloading _DAAC-hosted_ data but in this case the data are coming from the Earthdata Cloud.  For NASA data in the Earthdata Cloud, there is no charge to the user for egress from AWS Cloud servers.  This is not the case for other data in the cloud.

In [ ]:
granules = Query.get(4)
[display(g) for g in granules]

## Use Direct-Access to open, load and display data stored on S3

Direct-access to data from a S3 bucket, is a two step process.  First, the files are opened using the `open` method.  The `auth` object created at the start of the notebook is used to provide Earthdata Login authentication and AWS credentials.

The next step is to load the data.  In this case, data are loaded into an `xarray.Dataset`.  Data could be read into `numpy` arrays or a `pandas.Dataframe`.  However, each granule would have to be read using an package that reads HDF5 granules such as `h5py`.  `xarray` does this all _under-the-hood_ in a single line but for a single group in the HDF5 granule, in this case land ice heights for the gt1l beam*.

*ICESat-2 measures photon returns from 3 beam pairs numbered 1, 2 and 3 that each consist of a left and a right beam*

In [ ]:
%time
files = earthaccess.open(granules)
ds = xr.open_dataset(files[1], group='/gt1l/land_ice_segments')

In [ ]:
ds

`hvplot` is an interactive plotting tool that is useful for exploring data.

In [ ]:
ds['h_li'].hvplot(kind='scatter', s=2)

## **3. Learning outcomes recap**

We have learned how to:
1. use earthaccess to search for ICESat-2 data using spatial and temporal filters and explore search results;
2. open data granules using direct access to the ICESat-2 S3 bucket;
3. load a HDF5 group into an xarray.Dataset;
4. visualize the land ice heights using hvplot.

## **4. Additional resources**

For general information about NSIDC DAAC data in the Earthdata Cloud: 

[FAQs About NSIDC DAAC's Earthdata Cloud Migration](https://nsidc.org/data/user-resources/help-center/faqs-about-nsidc-daacs-earthdata-cloud-migration)

[NASA Earthdata Cloud Data Access Guide](https://nsidc.org/data/user-resources/help-center/nasa-earthdata-cloud-data-access-guide)

Additional tutorials and How Tos:

[NASA Earthdata Cloud Cookbook](https://nasa-openscapes.github.io/earthdata-cloud-cookbook/)